# Using the Qobuz API (`minim.qobuz`)

**Last updated**: June 12, 2023

In [1]:
from copy import deepcopy
import os
import pathlib
import sys

from IPython.display import display
import ipywidgets

sys.path.insert(0, f"{pathlib.Path(os.getcwd()).parents[2].resolve()}/src")
# The second argument above should be replaced with the path where you downloaded Minim.
from minim import qobuz

## Getting started

The `minim.qobuz` submodule offers procedural and object-oriented approaches to use the Qobuz API:

* The `minim.qobuz.Session` class, which is targeted at developers and experienced programmers, implements the Qobuz API exactly on top of the `requests` library. There are no bells and whistles; responses from API calls, if any, are in JavaScript Object Notation (JSON) format, and data extraction from or wrangling of the JSON data is done solely by the user.

* The `minim.qobuz.Album`, `minim.qobuz.Artist`, `minim.qobuz.Label`, `minim.qobuz.Playlist`, `minim.qobuz.Track`, and `minim.qobuz.User` classes are more user-friendly and are meant for beginners or hobbyists. Instead of having the user make API calls directly and deal with unpacking the JSON responses, these classes do all the heavy lifting and store the information returned by the API in instance attributes that have simple, logical names and are well-documented.

Regardless of which approach above you choose, it is recommended that you have a Qobuz account and, preferably, a Qobuz streaming plan to get access to all the API endpoints.

## As a first example

To compare the procedural and object-oriented approaches and as a first introductory example, let's search for and output basic information about the track "Post Malone (feat. RANI)" by Sam Feldt.

### Procedural approach

First, we create a `qobuz.Session` object without user authentication:

In [2]:
session_guest = qobuz.Session(authenticate=False)
print(session_guest)

Qobuz API: not logged in


As expected, the string representation of the `qobuz.Session` object confirms that no user authentication was performed. A consequence is that certain API endpoints are unavailable since they require either the user to be logged in or have an active Qobuz streaming plan.

Then, we search for tracks (or releases) with titles containing the phrase "Post Malone". For brevity, we limit our search results to only five items. The search results are in JSON format, with the first layer specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.) and the second the number of results for that media type and Qobuz catalog information for each match. We can preview the raw data for each track by iterating:

In [3]:
search_results = session_guest.search("Post Malone", "track", strict=True, limit=5)
for track in search_results["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '© 2019 SpinninRecords.com ℗ 2019 SpinninRecords.com', 'performers': 'Rani, FeaturedArtist - Roxanne Emery, Writer - Jordan Schultz, Masterer - Sam Feldt, Programmer, MainArtist - Sammy Renders, Writer - Dominic Lyttle, Mixer, Writer - Jonas Kröper, Writer - Shannon Hilversum, Writer', 'audio_info': {'replaygain_track_peak': 1, 'replaygain_track_gain': -10.44}, 'performer': {'name': 'Sam Feldt', 'id': 1829176}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_50.jpg', 'large': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 'Sam Feldt', 'id': 1829176, 'albums_count': 245, 'slug': 'sam-feldt', 'picture': None}, 'upc': '0190295418267', 'released_at': 1558648800, 'label': {'name': "Spinnin' Records", 'id': 190680, 'albums_count': 4

The first search result is the track we are looking for. We can extract the key details of the track by traversing through the `dict` containing the Qobuz catalog information for that track:

In [4]:
track_json_ex = search_results["tracks"]["items"][0]
print(f"Title: {track_json_ex['title']}")
print(f"Artist: {track_json_ex['performer']['name']}")
print(f"Album: {track_json_ex['album']['title']}")
print(f"Genre: {track_json_ex['album']['genre']['name']}")
print(f"Release date: {track['release_date_original']}")

Title: Post Malone (feat. RANI)
Artist: Sam Feldt
Album: Magnets EP
Genre: Dance
Release date: None


In some instances, the release date information is unavailable and will be `None`.

Now, let's get the producing and songwriting credits for the track:

In [5]:
credits = session_guest.get_track_credits(performers=track_json_ex["performers"])
for role, artists in credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

Mixer(s): Dominic Lyttle
Main Artist(s): Sam Feldt
Writer(s): Roxanne Emery, Sammy Renders, Dominic Lyttle, Jonas Kröper, Shannon Hilversum
Featured Artist(s): Rani
Programmer(s): Sam Feldt
Masterer(s): Jordan Schultz


What if we are building an application or widget to stream audio? We can get the URL of the track's stream using the track's Qobuz ID:

In [6]:
print(session_guest.get_track_file_url(track_json_ex["id"])["url"])

https://streaming-qobuz-std.akamaized.net/file?eid=61947440&fmt=6&profile=raw&range=20-30&app_id=950096963&etsp=1686560161&hmac=bDZz9SB6imfAIkTEjcA1lzNcRiI


Whoops! Because we didn't log into a Qobuz account with an active Qobuz streaming plan previously, we only got the 30-second preview of the track instead of the full audio. Let's authenticate ourselves using `qobuz.Session.login()` and try again. 

:::{note}
For the purposes of this tutorial, we will make a copy of the unauthenticated `qobuz.Session` object since we will reuse it in the upcoming sections. This is not necessary in normal use.
:::

:::{note}
No parameters are passed to `qobuz.Session.login()` below because an Qobuz user authentication token was found in my operating system's environment variable `QOBUZ_USER_AUTH_TOKEN`. If you have not stored user credentials or an authorization token in your OS's environment variables, you can pass either the `auth_token` argument or the `email` and `password` arguments to `qobuz.
:::

In [7]:
session = deepcopy(session_guest)
session.login()
stream_url = session.get_track_file_url(track_json_ex["id"])["url"]

:::{seealso}
We can also get the track's audio stream directly as a `bytes` object. See the [Working with tracks](#working-with-tracks) section below for more information.
:::

### Object-oriented approach

First, we create a `qobuz.User` object without user authentication:

In [8]:
user_guest = qobuz.User(authenticate=False)
print(user_guest)

Qobuz user: not logged in


As expected, the string representation of the `qobuz.User` object confirms that no user authentication was performed.

:::{note}
All objects spawned by the `qobuz.User` object shares the same Qobuz session information. Therefore, a `qobuz.Track` object created by the `qobuz.User` object above will also not be able to access information that requires user authentication. However, should the `qobuz.User` object be authenticated via `qobuz.User.login()`, the `qobuz.Track` object would also be authenticated.
:::

Like before, we search for tracks with titles containing the phrase "Post Malone", but now limit our search results to only five items. The search results are still contained in a `dict`, with the first key specifying the media type (`"albums"`, `"artists"`, `"tracks"`, etc.). However, the value corresponding to the `"tracks"` key is now a `list` of `minim.qobuz.Track` objects, each fully populated with that track's information.

In [9]:
search_results = user_guest.search("Post Malone", "track", strict=True, limit=5)
for track in search_results["tracks"]:
    print(track)

Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]
Qobuz track: 'Post Malone' [artist: Rowan Duy, album: Night Changes]
Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Post Malone (feat. RANI) [Joe Stone Remix]]
Qobuz track: 'Post Malone (feat. RANI) [VIZE Remix]' [artist: Sam Feldt, album: Post Malone (feat. RANI) [VIZE Remix]]
Qobuz track: 'Post Malone (feat. RANI) [GATTÜSO Remix]' [artist: Sam Feldt, album: Post Malone (feat. RANI) [The Remixes]]


With human-readable string representations of each track, it is easy to see that the first search result is the track we are looking for. We can extract the key details of the track by getting instance attributes of the `minim.qobuz.Track` object:

In [10]:
track_obj_ex = search_results["tracks"][0]
print(f"Title: {track_obj_ex.title}")
print(f"Artist: {track_obj_ex.main_artist.name}")
print(f"Album: {track_obj_ex.album.title}")
print(f"Genre: {track_obj_ex.album.genre}")
print(f"Release date: {track_obj_ex.release_date}")

Title: Post Malone (feat. RANI)
Artist: Sam Feldt
Album: Magnets EP
Genre: Dance
Release date: None


Now, all data in the `minim.qobuz.Track` object is human-readable, even those dealing with datetimes. Furthermore, the producing and songwriting credits for the track are already embedded in the `minim.qobuz.Track` object:

In [11]:
for role, artists in track_obj_ex.credits.items():
    print(f"{' '.join(role.split('_')).title()}(s): {', '.join(artists)}")

Mixer(s): Dominic Lyttle
Main Artist(s): Sam Feldt
Writer(s): Roxanne Emery, Sammy Renders, Dominic Lyttle, Jonas Kröper, Shannon Hilversum
Featured Artist(s): Rani
Programmer(s): Sam Feldt
Masterer(s): Jordan Schultz


Getting the audio stream data no longer requires knowledge of the track ID:

In [12]:
track_obj_ex.get_file_url()
print(track_obj_ex.file_url)

https://streaming-qobuz-std.akamaized.net/file?eid=61947440&fmt=6&profile=raw&range=20-30&app_id=950096963&etsp=1686560185&hmac=bY9UKmwvY9xYukW70SEqBJbR1I4


Oops, forgot about user authentication again. This time, we authenticate using `minim.Qobuz.User.login()` instead.

:::{note}
For the purposes of this tutorial, we will make a copy of the unauthenticated `qobuz.User` object since we will reuse it in the upcoming sections. This is not necessary in normal use.
:::

In [13]:
user = deepcopy(user_guest)
user.login()
track_obj_ex.get_file_url()

:::{seealso}
We can also get the track's audio stream directly as a `bytes` object. See the [Working with tracks](#working-with-tracks) section below for more information.
:::

## Working with artists

An artist is a person or group that performs, appears on, or contributed to an album or track. Every album, playlist, and track contains artist information in some form.

In the following section, we will retrieve information about an artist (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will focus on the Dutch EDM duo Sick Individuals.

In [14]:
artist_id = 764370

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the artist. To include albums, playlists, and tracks that the artist appears on, we pass `extras=["albums", "playlists", "tracks"]` to `qobuz.Session.get_artist()`. For brevity, we limit our search results to only five items.

In [15]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

artist_json = session_guest.get_artist(artist_id, extras=["albums", "playlists", "tracks"], limit=5)
for key, value in artist_json.items():
    print(key, value, sep=": ")

id: 764370
name: Sick Individuals
albums_as_primary_artist_count: 146
albums_as_primary_composer_count: 0
albums_count: 121
slug: sick-individuals
picture: None
image: {'small': 'https://static.qobuz.com/images/artists/covers/small/7e0606708c0ef65462c5311fe81099de.jpg', 'medium': 'https://static.qobuz.com/images/artists/covers/medium/7e0606708c0ef65462c5311fe81099de.jpg', 'large': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg', 'extralarge': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg', 'mega': 'https://static.qobuz.com/images/artists/covers/large/7e0606708c0ef65462c5311fe81099de.jpg'}
similar_artist_ids: [67661, 97536, 725598, 2103126, 301564, 720916, 955835, 1171943, 798810, 974660, 1048406]
information: None
biography: {'summary': 'Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, 

We can see that the artist's albums, playlists, and tracks are stored in their respective keys above.

The only other interesting piece of information returned is a short biography of the artist:

In [16]:
print(artist_json['biography']['content'].split('\n')[0])

Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, who met in 2008 while studying music composition. Both individuals already had experience penning music for commercials, but they bonded over a mutual love for acts like Daft Punk and Swedish House Mafia, and decided to make dance music together. Sick Individuals began releasing original tracks on Sneakerz Muzik in 2010, and were initially more prolific as remixers, with clients including Flo Rida, Nicky Romero, and Freestylers. Their mix of Icona Pop's 2012 smash hit "I Love It" was particularly well received, and the duo followed it with numerous singles on labels like Revealed Recordings, Axtone, Onelove, and Armada. Collaborations with Dannic ("Blueprint") and Axwell ("I Am") both topped charts of dance music retailer Beatport, and the duo remixed songs by Madonna, Rihanna, Lady Gaga, Avicii, and numerous other stars. Sic

### Object-oriented approach

First, we directly create a `qobuz.Artist` object using the artist's Qobuz ID. We also request albums, playlists, and tracks that the artist contributed to by including the `extras=["albums", "playlists", "tracks"]` parameter. Information about the artist is automatically retrieved.

In [17]:
artist_obj = qobuz.Artist(artist_id, extras=["albums", "playlists", "tracks"], limit=5, authenticate=False)
print(artist_obj)

Qobuz artist: Sick Individuals


Below are the albums, playlists, and tracks that were requested:

In [18]:
for album in artist_obj.albums:
    print(album)
for playlist in artist_obj.playlists:
    print(playlist)
for track in artist_obj.tracks:
    print(track)

Qobuz album: 'We Go Out' [artist: Alesso]
Qobuz album: 'My Life (feat. The Runaway Club)' [artist: THE RUNAWAY CLUB]
Qobuz album: 'With My Friends' [artist: Tungevaag]
Qobuz album: 'Miss You' [artist: Tungevaag]
Qobuz album: 'Feel Your Love' [artist: Dannic]
Qobuz playlist: 'Sick Individuals' [owner: Qobuz]
Qobuz track: 'MAIA' [artist: Sick Individuals, album: MAIA]
Qobuz track: 'Heart On Fire' [artist: Sick Individuals, album: Heart On Fire]
Qobuz track: 'YOU (feat. Dotter)' [artist: Sick Individuals, album: YOU (feat. Dotter)]
Qobuz track: 'My Life (feat. The Runaway Club)' [artist: Sick Individuals, album: My Life (feat. The Runaway Club)]
Qobuz track: 'Better with You' [artist: Sick Individuals, album: Better with You]


Like before, a brief summary of the artist is available:

In [19]:
print(artist_obj.biography.split("\n")[0])

Dutch EDM duo Sick Individuals write boisterous, catchy dance tunes equally suited for festivals and the radio. The duo consists of Joep Smeele and Rinze Hofstee, who met in 2008 while studying music composition. Both individuals already had experience penning music for commercials, but they bonded over a mutual love for acts like Daft Punk and Swedish House Mafia, and decided to make dance music together. Sick Individuals began releasing original tracks on Sneakerz Muzik in 2010, and were initially more prolific as remixers, with clients including Flo Rida, Nicky Romero, and Freestylers. Their mix of Icona Pop's 2012 smash hit "I Love It" was particularly well received, and the duo followed it with numerous singles on labels like Revealed Recordings, Axtone, Onelove, and Armada. Collaborations with Dannic ("Blueprint") and Axwell ("I Am") both topped charts of dance music retailer Beatport, and the duo remixed songs by Madonna, Rihanna, Lady Gaga, Avicii, and numerous other stars. Sic

:::{seealso}
While methods like `qobuz.Artist.favorite()` and `qobuz.Artist.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with tracks

A track (or a release) is the most central Qobuz object. Tracks are a key part of albums and playlists and can be associated with artists who appear or feature on them.

In the following section, we will retrieve information about a track (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the track "Hold On & Believe (feat. The Federal Empire)" by Martin Garrix.

In [20]:
track_id = 35560953

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the track:

In [21]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

track_json = session_guest.get_track(track_id)
for key, value in track_json.items():
    print(key, value, sep=": ")

maximum_bit_depth: 24
copyright: (P) 2016 STMPD RCRDS B.V. exclusively licensed to Epic Amsterdam, a divison of Sony Music Entertainment Netherlands B.V.
 - Chad Wolf, Composer, Lyricistcististte, Associated Performer Artist, Producer
audio_info: {'replaygain_track_gain': -11.15, 'replaygain_track_peak': 0.965027}
performer: {'id': 974660, 'name': 'Martin Garrix'}
album: {'maximum_bit_depth': 24, 'image': {'small': 'https://static.qobuz.com/images/covers/03/98/0886446179803_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/03/98/0886446179803_50.jpg', 'large': 'https://static.qobuz.com/images/covers/03/98/0886446179803_600.jpg', 'back': None}, 'media_count': 1, 'artist': {'image': None, 'name': 'Martin Garrix', 'id': 974660, 'albums_count': 381, 'slug': 'martin-garrix', 'picture': None}, 'artists': [{'id': 974660, 'name': 'Martin Garrix', 'roles': ['main-artist']}, {'id': 2770110, 'name': 'The Federal Empire', 'roles': ['featured-artist']}], 'upc': '0886446179803', 'releas

The standard metadata for the track is listed below:

In [22]:
print(f"Track: {track_json['track_number']}/{track_json['album']['tracks_count']}")
print(f"Disc: {track_json['media_number']}/{track_json['album']['media_count']}")
print(f"Title: {track_json['title']}")
print(f"Artist: {track_json['performer']['name']}")
print(f"Album: {track_json['album']['title']}")
print(f"Album artist: {track_json['album']['artist']['name']}")
print(f"Genre: {track_json['album']['genre']['name']}")
print(f"Release date: {track_json['release_date_original']}")
print(f"Copyright: {track_json['copyright']}")
print(f"ISRC: {track_json['isrc']}")

Track: 1/1
Disc: 1/1
Title: Hold On & Believe
Artist: Martin Garrix
Album: Hold On & Believe
Album artist: Martin Garrix
Genre: House
Release date: None
Copyright: (P) 2016 STMPD RCRDS B.V. exclusively licensed to Epic Amsterdam, a divison of Sony Music Entertainment Netherlands B.V.
ISRC: NLM5S1600010


:::{note}
`track["title"]` often does not list the featured artists or contain additional information about the track (such as "KSHMR Remix", "2011 Remastered", "Taylor's Version", etc.). The featured artists can only be found in the track credits (see below), while the track version information, if any, is contained in `track["version"]`.
:::

`track["performer"]` only contains information about the primary artist associated with the track and can be incomplete. The full list of contributors can be found in `track["performers"]`:

In [23]:
print(" -", track_json["performers"].replace(" - ", "\n - "))

 - Martin Garrix, Associated Performer, Engineer, Main Artist, Producer
 - The Federal Empire, Associated Performer, Featured Artist
 - Martin Garrix Feat. The Federal Empire, Associated Performer
 - Martijn Garritsen, Composer, Lyricist
 - Giorgio Tuinfort, Composer, Lyricist
 - Simon Strömstedt, Composer, Lyricist
 - Keith Varon, Composer, Lyricist
 - McKay Stevens, Composer, Lyricist
 - Chad Wolf, Composer, Lyricist


The track credits above, but categorized by role, can be obtained by passing `performers=track["performers"]` (or `track_id` directly) to `qobuz.Session.get_track_credits()`:

In [24]:
for key, value in session_guest.get_track_credits(performers=track_json["performers"]).items():
    print(key, value, sep=": ")

lyricist: ['Martijn Garritsen', 'Giorgio Tuinfort', 'Simon Strömstedt', 'Keith Varon', 'McKay Stevens', 'Chad Wolf']
associated_performer: ['Martin Garrix', 'The Federal Empire', 'Martin Garrix Feat. The Federal Empire']
composer: ['Martijn Garritsen', 'Giorgio Tuinfort', 'Simon Strömstedt', 'Keith Varon', 'McKay Stevens', 'Chad Wolf']
featured_artist: ['The Federal Empire']
main_artist: ['Martin Garrix']
producer: ['Martin Garrix']
engineer: ['Martin Garrix']


For streaming purposes, the file URL for the track's audio stream can be retrieved using `qobuz.Session.get_track_file_url()`. The Qobuz ID is the first (required) positional argument. The audio quality of the audio stream can optionally be specified using the `quality` argument, with the following possible values:

* `5` or `"MP3"` for constant bitrate (320 kbps) MP3
* `6` or `"CD"` for CD-quality (16-bit, 44.1 kHz) FLAC
* `7` for up to 24-bit, 96 kHz Hi-Res FLAC
* default: `27` or `"HI-RES"` for up to 24-bit, 192 kHz Hi-Res FLAC

In [25]:
file_url = session_guest.get_track_file_url(track_id, quality="MP3")
for key, value in file_url.items():
    print(key, value, sep=": ")

track_id: 35560953
duration: 30
url: https://streaming-qobuz-std.akamaized.net/file?eid=35560953&fmt=7&profile=raw&range=20-30&app_id=950096963&etsp=1686560223&hmac=45x1S-YRbzhTgJde5nwdDK-VIZQ
format_id: 7
mime_type: audio/flac
sample: True
restrictions: [{'code': 'UserUnauthenticated'}]
sampling_rate: 44.1
bit_depth: 24


For applications where the raw audio stream in `bytes` is necessary, we can use `qobuz.Session.get_track_stream()` with the same arguments instead:

In [26]:
ipywidgets.Audio(value=session_guest.get_track_stream(track_id, quality="MP3"), autoplay=False)

Audio(value=b'\xff\xfb\xe0d\xfc\x82\x0bJx\xd0\xab\x9a\xcb\xc2\xc1+\x9a\xcdg\x0c\\c\xe1\xf3U\x8d\xe5\x91\xca\xa…

### Object-oriented approach

First, we directly create a `qobuz.Track` object using the track's Qobuz ID. Information about the track is automatically retrieved.

In [27]:
track_obj = qobuz.Track(track_id, authenticate=False)
print(track_obj)

Qobuz track: 'Hold On & Believe (feat. The Federal Empire)' [artist: Martin Garrix, album: Hold On & Believe]


Like before, the standard metadata for the track is listed below:

In [28]:
print(f"Track: {track_obj.track_number}/{track_obj.album.track_count}")
print(f"Disc: {track_obj.disc_number}/{track_obj.album.disc_count}")
print(f"Title: {track_obj.title}")
print(f"Artist: {track_obj.main_artist.name}")
print(f"Album: {track_obj.album.title}")
print(f"Album artist: {track_obj.album.primary_artist.name}")
print(f"Genre: {track_obj.album.genre}")
print(f"Release date: {track_obj.release_date}")
print(f"Copyright: {track_obj.copyright}")
print(f"ISRC: {track_obj.isrc}")

Track: 1/1
Disc: 1/1
Title: Hold On & Believe (feat. The Federal Empire)
Artist: Martin Garrix
Album: Hold On & Believe
Album artist: Martin Garrix
Genre: House
Release date: None
Copyright: (P) 2016 STMPD RCRDS B.V. exclusively licensed to Epic Amsterdam, a divison of Sony Music Entertainment Netherlands B.V.
ISRC: NLM5S1600010


Unlike before, the `qobuz.Track.title` attribute will always have information about the featured artists and track version, if available. Additional pre-processing is automatically done to ensure this, but a potential downside is that the full title is unmanageably long (like "We Are The People (feat. Bono & The Edge) [Martin Garrix Remix] [Official UEFA EURO 2020 Song]", which clocks in at *94 characters long*).

The track credits is only available pre-formatted:

In [29]:
for key, value in track_obj.credits.items():
    print(key, value, sep=": ")

lyricist: ['Martijn Garritsen', 'Giorgio Tuinfort', 'Simon Strömstedt', 'Keith Varon', 'McKay Stevens', 'Chad Wolf']
associated_performer: ['Martin Garrix', 'The Federal Empire', 'Martin Garrix Feat. The Federal Empire']
composer: ['Martijn Garritsen', 'Giorgio Tuinfort', 'Simon Strömstedt', 'Keith Varon', 'McKay Stevens', 'Chad Wolf']
featured_artist: ['The Federal Empire']
main_artist: ['Martin Garrix']
producer: ['Martin Garrix']
engineer: ['Martin Garrix']


Finally, the file URL and stream data can be obtained by calling methods innate to the `qobuz.Track` object:

In [30]:
track_obj.get_file_url(quality="MP3")
print(track_obj.file_url)

https://streaming-qobuz-std.akamaized.net/file?eid=35560953&fmt=7&profile=raw&range=20-30&app_id=950096963&etsp=1686560234&hmac=d9ODbyZ9XDo8b_-6e9iSLUe57PQ


Or, if the audio stream in `bytes` is desired:

In [31]:
track_obj.get_stream(quality="MP3")
ipywidgets.Audio(value=track_obj.stream, autoplay=False)

Audio(value=b'\xff\xfb\xe0d\xfc\x82\x0bJx\xd0\xab\x9a\xcb\xc2\xc1+\x9a\xcdg\x0c\\c\xe1\xf3U\x8d\xe5\x91\xca\xa…

:::{seealso}
While methods like `qobuz.Track.favorite()` and `qobuz.Track.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with albums

Albums, including extended plays (EPs) and singles, are collections of tracks by a primary artist. Every track belongs to an album.

In the following section, we will retrieve information about an album (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the album 'Field Trip' by NIIKO X SWAE.

In [32]:
album_id = "cdgt6fwyvhz3a"

### Procedural approach

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the album:

In [33]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

album_json = session_guest.get_album(album_id)
for key, value in album_json.items():
    print(key, value, sep=": ")

maximum_bit_depth: 16
image: {'small': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_50.jpg', 'large': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_600.jpg', 'back': None}
media_count: 1
artist: {'image': None, 'name': 'Niiko x SWAE', 'id': 3327805, 'albums_count': 63, 'slug': 'niikoxswae-10003327805', 'picture': None}
artists: [{'id': 3327805, 'name': 'Niiko x SWAE', 'roles': ['main-artist']}]
upc: 8718522342165
released_at: 1623362400
label: {'name': 'Armada Music', 'id': 248166, 'albums_count': 4336, 'supplier_id': 41, 'slug': 'armada-music'}
title: Field Trip
qobuz_id: 121976829
version: None
url: https://www.qobuz.com/fr-fr/album/field-trip-niiko-x-swae/cdgt6fwyvhz3a
duration: 726
popularity: 0
tracks_count: 4
genre: {'path': [64, 129], 'color': '#5eabc1', 'name': 'Dance', 'id': 129, 'slug': 'dance'}
maximum_channel_count: 2
id: cdgt6fwyvhz3a
maximum_sampling_rate: 44.1


Here is some basic information about the album:

In [34]:
print(f"Title: {album_json['title']}")
print(f"Artist: {album_json['artist']['name']}")
print(f"Genre: {album_json['genre']['name']}")
print(f"Release date: {album_json['release_date_original']}")
print(f"Copyright: {album_json['copyright']}")

Title: Field Trip
Artist: Niiko x SWAE
Genre: Dance
Release date: 2021-06-11
Copyright: 2021 Armada Music B.V. 2021 Armada Music B.V.


The JSON data also contains information about the tracks in the album:

In [35]:
for track in album_json["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '2021 Armada Music B.V. 2021 Armada Music B.V.', 'performers': 'Gazzo, Producer - April Bender, Composer, Lyricist, FeaturedArtist - Nikolaus Becker, Composer - Lena Leon, Composer, Lyricist - Michael Gazzo, Composer - Niiko x SWAE, Producer, MainArtist - Mark Manchester, Composer - Ultra Empire Music (BMI) obo itself and LRL Music (BMI), MusicPublisher - Niiko x SWAE Publishing, MusicPublisher - Torro Rosso Tracks, MusicPublisher - ARCHERSONGS Publishing (ASCAP), MusicPublisher', 'audio_info': {'replaygain_track_peak': 0.98877, 'replaygain_track_gain': -11.33}, 'performer': {'name': 'Niiko x SWAE', 'id': 3327805}, 'work': None, 'composer': {'name': 'Nikolaus Becker', 'id': 1758157}, 'isrc': 'NLF712100910', 'title': "I Ain't Going Home", 'version': None, 'duration': 182, 'parental_warning': False, 'track_number': 1, 'maximum_channel_count': 2, 'id': 121976830, 'media_number': 1, 'maximum_sampling_rate': 44.1, 'release_date_original': None, 'releas

Having access to the track IDs allows for track operations, such as getting the file URLs (or audio stream data) for streaming purposes:

In [36]:
for track in album_json["tracks"]["items"]:
    print(session_guest.get_track_file_url(track["id"], quality="MP3")["url"])
    display(ipywidgets.Audio(value=session_guest.get_track_stream(track["id"], quality="MP3"), autoplay=False))

https://streaming-qobuz-std.akamaized.net/file?eid=121976830&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560238&hmac=NVQR4plXA_yvjZeZElL0OVHjm0A


Audio(value=b"\xff\xfb\xe2D\xf5\x03'\xcdf\xd2\xf3:}3\x07\xcd\x1a^i\xe6\xe8_M\x99M\xcc\xe9\xf4\xcb\xf74\xa9\xfd…

https://streaming-qobuz-std.akamaized.net/file?eid=121976831&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560239&hmac=mFcvgNCqLYwTDiSrZeR0Whbi0j8


Audio(value=b'\xff\xfb\xe2D\xe8\x02(\xf6iR\xc3y}s\t\xed*\xado/\x8e\x1e)\xa1S\x8d=\x9a\xcb\xdd5*\x05\xbc>:\x844…

https://streaming-qobuz-std.akamaized.net/file?eid=121976832&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560239&hmac=yFlOnpd6nXHfNt2fJF-L6HwrV8o


Audio(value=b"\xff\xfb\xe2d\xf5\x02\n'mPC\xb9\xc2\xa0\xb4\xaa*Ti\xe9\x9a%\xcd\xb7B\x8ek\x07J\xcd.i\xb5\xa7\x9e…

https://streaming-qobuz-std.akamaized.net/file?eid=121976833&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560240&hmac=aMFboZu9FWV_9f2ezNHwQd0k5xM


Audio(value=b'\xff\xfb\xe2D\xfd\x84(SuT[OOC\x01\x8e\xfa\x98i\xe6\xe8\x1e\xa9\xddUM<Y\xc4\n\xba*\x15\xb7\x9b\xa…

### Object-oriented approach

First, we directly create a `qobuz.Album` object using the album's Qobuz ID. 

In [37]:
album_obj = qobuz.Album(album_id, authenticate=False)
print(album_obj)

Qobuz album: 'Field Trip' [artist: Niiko x SWAE]


Now, information about the album is stored in the `qobuz.Album` object's instance attributes:

In [38]:
print(f"Title: {album_obj.title}")
print(f"Artist: {album_obj.primary_artist}")
print(f"Genre: {album_obj.genre}")
print(f"Release date: {album_obj.release_date}")
print(f"Copyright: {album_obj.copyright}")

Title: Field Trip
Artist: Qobuz artist: Niiko x SWAE
Genre: Dance
Release date: 2021-06-11 00:00:00
Copyright: 2021 Armada Music B.V. 2021 Armada Music B.V.


Information about the tracks in the album are stored in `qobuz.Track` objects:

In [39]:
for track in album_obj.tracks:
    print(track)

Qobuz track: 'I Ain't Going Home (feat. April Bender)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Glue (feat. Kyle Reynolds)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Better People (feat. Max Embers)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Together Again' [artist: Niiko x SWAE, album: Field Trip]


To get the track's file URLs or streams, we can either iterate through each `qobuz.Track` object and call its `qobuz.Track.get_file_url()` or `qobuz.Track.get_stream()` methods, or use the convenient `qobuz.Album.get_file_urls()` or `qobuz.Album.get_streams()` methods.

In [40]:
album_obj.get_file_urls(quality="MP3")
album_obj.get_streams(quality="MP3")
for track in album_obj.tracks:
    print(track.file_url)
    display(ipywidgets.Audio(value=track.stream, autoplay=False))

https://streaming-qobuz-std.akamaized.net/file?eid=121976830&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560259&hmac=oHQnr8KoFWP9uTRmSmAFszpR1ns


Audio(value=b"\xff\xfb\xe2D\xf5\x03'\xcdf\xd2\xf3:}3\x07\xcd\x1a^i\xe6\xe8_M\x99M\xcc\xe9\xf4\xcb\xf74\xa9\xfd…

https://streaming-qobuz-std.akamaized.net/file?eid=121976831&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560259&hmac=ZTHAiat6kX5mXBKNc4ectxpzeYI


Audio(value=b'\xff\xfb\xe2D\xe8\x02(\xf6iR\xc3y}s\t\xed*\xado/\x8e\x1e)\xa1S\x8d=\x9a\xcb\xdd5*\x05\xbc>:\x844…

https://streaming-qobuz-std.akamaized.net/file?eid=121976832&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560259&hmac=-Q6AF9tbGkJ7gyj3vKN1rYRevy0


Audio(value=b"\xff\xfb\xe2d\xf5\x02\n'mPC\xb9\xc2\xa0\xb4\xaa*Ti\xe9\x9a%\xcd\xb7B\x8ek\x07J\xcd.i\xb5\xa7\x9e…

https://streaming-qobuz-std.akamaized.net/file?eid=121976833&fmt=5&profile=raw&range=20-30&app_id=950096963&etsp=1686560259&hmac=oIyaOtQKkmEZY9RHIATRoBKshXA


Audio(value=b'\xff\xfb\xe2D\xfd\x84(SuT[OOC\x01\x8e\xfa\x98i\xe6\xe8\x1e\xa9\xddUM<Y\xc4\n\xba*\x15\xb7\x9b\xa…

:::{seealso}
While methods like `qobuz.Album.favorite()` and `qobuz.Album.unfavorite()` are available, it is recommended that `qobuz.User.favorite_items()` and `qobuz.User.unfavorite_items()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

## Working with playlists

Playlists are curated collections of tracks. There are two kinds: playlists curated by Qobuz that predominantly feature tracks of the same genre or by related artists, or personal playlists created by Qobuz users.

In the following section, we will retrieve information about a public playlist (for which we know the Qobuz ID for) using both the procedural and object-oriented approaches. For this tutorial, we will use the curated playlist 'Taylor Swift' consisting of her songs.

In [41]:
playlist_id = 2512796

### Procedural approach for public playlists

In this section, public playlists refer to playlists owned by Qobuz or other Qobuz users. Even with user authentication, you cannot edit these playlists because they do not belong to you. You can only query information about the playlists and the tracks in them.

First, we create a `qobuz.Session` object without user authentication and use it to retrieve Qobuz catalog information for the playlist:

In [42]:
# session_guest = qobuz.Session(authenticate=False)
# We are reusing the unauthenticated qobuz.Session object from before.

playlist_json = session_guest.get_playlist(playlist_id)
for key, value in playlist_json.items():
    print(key, value, sep=": ")

image_rectangle_mini: ['https://static.qobuz.com/images/playlists/2512796_f6ee8023026e325dfe5a44eaa9562e2b_rectangle_mini.jpg']
featured_artists: [{'id': 70899, 'name': 'Taylor Swift', 'slug': 'taylor-swift', 'albums_count': 978, 'picture': None, 'image': None}]
description: With 50 million albums sold worldwide, the 29-year-old Taylor Swift is now one of the top record sellers of all time. Since her debut in 2006, she’s always had a keen sense of when and how to reinvent herself, departing from her roots in country music to a calibrated but particularly well-constructed brand of pop music. Never fearful to express herself with the utmost freedom, Swift has always been an independent artist at heart. Her music is intelligent and unifying and has made the American a formidable hit machine with plenty of spirit.

Photo © Taylor Swift
created_at: 1570648837
timestamp_position: 1670938157
images300: ['https://static.qobuz.com/images/covers/xb/2u/h1r19zw2l2uxb_300.jpg', 'https://static.qobu

Just like albums, playlists contain tracks. Here is the JSON information for the tracks:

In [43]:
for track in playlist_json["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '℗ 2014 Big Machine Records, LLC', 'performers': 'Max Martin, Producer, ComposerLyricist - Shellback, Producer, ComposerLyricist - Taylor Swift, MainArtist, ComposerLyricist', 'audio_info': {'replaygain_track_peak': 0.99884, 'replaygain_track_gain': -9.84}, 'performer': {'name': 'Taylor Swift', 'id': 70899}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/ta/ym/id29re6slymta_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/ta/ym/id29re6slymta_50.jpg', 'large': 'https://static.qobuz.com/images/covers/ta/ym/id29re6slymta_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 'Taylor Swift', 'id': 70899, 'albums_count': 952, 'slug': 'taylor-swift', 'picture': None}, 'upc': '0843930013609', 'released_at': 1414364400, 'label': {'name': 'Big Machine Records, LLC', 'id': 17893, 'albums_count': 985, 'supplier_id': 1, 'slug': 'big-machine-records-llc'}, 'title': '1989', 'qobuz_id': 5396534

:::{seealso}
All track-related playlist operations are the same as those for albums. See the [Working with albums](#working-with-albums) section above for more information.
:::

### Object-oriented approach for public playlists

First, we directly create a `qobuz.Playlist` object using the playlist's Qobuz ID. 

In [44]:
playlist_obj = qobuz.Playlist(playlist_id, authenticate=False)
print(playlist_obj)

Qobuz playlist: 'Taylor Swift' [owner: Qobuz]


Just like albums, information about the tracks in the playlist are stored in `qobuz.Track` objects:

In [45]:
for track in playlist_obj.tracks:
    print(track)

Qobuz track: 'Shake It Off' [artist: Taylor Swift, album: 1989 (Deluxe Edition)]
Qobuz track: 'We Are Never Ever Getting Back Together (Taylor's Version)' [artist: Taylor Swift, album: Red (Taylor's Version)]
Qobuz track: 'You Need To Calm Down' [artist: Taylor Swift, album: Lover]
Qobuz track: 'Anti-Hero' [artist: Taylor Swift, album: Midnights]
Qobuz track: 'Love Story (Taylor’s Version)' [artist: Taylor Swift, album: Fearless (Taylor's Version)]
Qobuz track: 'willow' [artist: Taylor Swift, album: evermore (Explicit)]
Qobuz track: 'I Knew You Were Trouble (Taylor's Version)' [artist: Taylor Swift, album: Red (Taylor's Version)]
Qobuz track: 'cardigan' [artist: Taylor Swift, album: Folklore (Explicit)]
Qobuz track: 'I Bet You Think About Me (Taylor's Version) (From The Vault) [feat. Chris Stapleton]' [artist: Taylor Swift, album: Red (Taylor's Version)]
Qobuz track: 'Blank Space' [artist: Taylor Swift, album: 1989 (Deluxe Edition)]
Qobuz track: 'Teardrops On My Guitar (Radio Single Re

:::{seealso}
All track-related playlist operations are the same as those for `qobuz.Album` objects. See the [Working with albums](#working-with-albums) section above for more information.
:::

:::{seealso}
While methods like `qobuz.Playlist.favorite()` and `qobuz.Playlist.unfavorite()` are available, it is recommended that `qobuz.User.favorite_playlist()` and `qobuz.User.unfavorite_playlist()` are used instead. See the [Account- and user-related methods](#account--and-user-related-methods) section below for more information.
:::

### Procedural approach for custom playlists

Most, if not all, custom user playlist endpoints require user authentication. As such, we now start with a `qobuz.Session` object with user authentication.

:::{note}
No parameters are passed to `qobuz.Session.login()` below because an Qobuz user authentication token was found in my operating system's environment variable `QOBUZ_USER_AUTH_TOKEN`. If you have not stored user credentials or an authorization token in your OS's environment variables, you can pass either the `auth_token` argument or the `email` and `password` arguments to `qobuz.
:::

In [46]:
# session = qobuz.Session()

To create a playlist, we use `qobuz.Session.create_playlist()` and pass in the playlist's name as the first argument. Optionally, we can also give a description and boolean flags to determine whether the playlist is public and collaborative.

In [47]:
user_playlist_json = session.create_playlist("Minim", description="An example custom user playlist.")
for key, value in user_playlist_json.items():
    if key != "owner":
        print(key, value, sep=": ")

id: 15552033
name: Minim
description: An example custom user playlist.
tracks_count: 0
users_count: 0
duration: 0
public_at: 1686520800
created_at: 1686556711
updated_at: 1686556711
is_public: True
is_collaborative: False


In case we want to update the playlist's properties, we can use `qobuz.Session.update_playlist()`. For example, to make the playlist private, we can do:

In [48]:
user_playlist_json = session.update_playlist(user_playlist_json["id"], public=False)
for key, value in user_playlist_json.items():
    if key != "owner":
        print(key, value, sep=": ")

id: 15552033
name: Minim
description: An example custom user playlist.
tracks_count: 0
users_count: 0
duration: 0
public_at: False
created_at: 1686556711
updated_at: 1686556712
is_public: False
is_collaborative: False


Let's add some tracks to the playlist using `qobuz.Session.add_playlist_tracks()`. Afterwards, we use `qobuz.Session.get_playlist()` to query the tracks we just added.

In [49]:
session.add_playlist_tracks(user_playlist_json["id"], 
                            [track_json_ex["id"], 
                             track_json["id"], 
                             album_json["tracks"]["items"][0]["id"],
                             playlist_json["tracks"]["items"][14]["id"]])
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '© 2019 SpinninRecords.com ℗ 2019 SpinninRecords.com', 'performers': 'Rani, FeaturedArtist - Roxanne Emery, Writer - Jordan Schultz, Masterer - Sam Feldt, Programmer, MainArtist - Sammy Renders, Writer - Dominic Lyttle, Mixer, Writer - Jonas Kröper, Writer - Shannon Hilversum, Writer', 'audio_info': {'replaygain_track_peak': 1, 'replaygain_track_gain': -10.44}, 'performer': {'name': 'Sam Feldt', 'id': 1829176}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_50.jpg', 'large': 'https://static.qobuz.com/images/covers/cc/09/tfjs3torp09cc_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 'Sam Feldt', 'id': 1829176, 'albums_count': 245, 'slug': 'sam-feldt', 'picture': None}, 'upc': '0190295418267', 'released_at': 1558648800, 'label': {'name': "Spinnin' Records", 'id': 190680, 'albums_count': 4

To reorder tracks in the playlist, use `qobuz.Session.move_playlist_tracks()`. Let's move the last track to the second position.

:::{note}
The argument `playlist_track_ids` should be the playlist track ID (which is specific to a track in a given playlist), not the Qobuz track ID.
:::

In [50]:
session.move_playlist_tracks(user_playlist_json["id"], 
                             user_playlist_json["tracks"]["items"][2]["playlist_track_id"], 
                             0)
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '2021 Armada Music B.V. 2021 Armada Music B.V.', 'performers': 'Gazzo, Producer - April Bender, Composer, Lyricist, FeaturedArtist - Nikolaus Becker, Composer - Lena Leon, Composer, Lyricist - Michael Gazzo, Composer - Niiko x SWAE, Producer, MainArtist - Mark Manchester, Composer - Ultra Empire Music (BMI) obo itself and LRL Music (BMI), MusicPublisher - Niiko x SWAE Publishing, MusicPublisher - Torro Rosso Tracks, MusicPublisher - ARCHERSONGS Publishing (ASCAP), MusicPublisher', 'audio_info': {'replaygain_track_peak': 0.98877, 'replaygain_track_gain': -11.33}, 'performer': {'name': 'Niiko x SWAE', 'id': 3327805}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_50.jpg', 'large': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 

To delete tracks from the playlist, use `qobuz.Session.delete_playlist_tracks()`. Let's remove the third track from the playlist.

In [51]:
session.delete_playlist_tracks(user_playlist_json["id"], 
                               user_playlist_json["tracks"]["items"][2]["playlist_track_id"])
user_playlist_json = session.get_playlist(user_playlist_json["id"])
for track in user_playlist_json["tracks"]["items"]:
    print(track)

{'maximum_bit_depth': 16, 'copyright': '2021 Armada Music B.V. 2021 Armada Music B.V.', 'performers': 'Gazzo, Producer - April Bender, Composer, Lyricist, FeaturedArtist - Nikolaus Becker, Composer - Lena Leon, Composer, Lyricist - Michael Gazzo, Composer - Niiko x SWAE, Producer, MainArtist - Mark Manchester, Composer - Ultra Empire Music (BMI) obo itself and LRL Music (BMI), MusicPublisher - Niiko x SWAE Publishing, MusicPublisher - Torro Rosso Tracks, MusicPublisher - ARCHERSONGS Publishing (ASCAP), MusicPublisher', 'audio_info': {'replaygain_track_peak': 0.98877, 'replaygain_track_gain': -11.33}, 'performer': {'name': 'Niiko x SWAE', 'id': 3327805}, 'album': {'image': {'small': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_230.jpg', 'thumbnail': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_50.jpg', 'large': 'https://static.qobuz.com/images/covers/3a/hz/cdgt6fwyvhz3a_600.jpg'}, 'maximum_bit_depth': 16, 'media_count': 1, 'artist': {'image': None, 'name': 

To delete the whole playlist, use `qobuz.Session.delete_playlist()`:

In [52]:
session.delete_playlist(user_playlist_json["id"])

### Object-oriented approach for custom playlists

To create a Qobuz playlist, we can instantiate a `qobuz.Playlist` object with the first argument being a `dict` with the arguments to pass to `qobuz.Session.create_playlist()`. To grant permissions to the object, we can pass in keyword arguments containing either an authenticated `qobuz.Session` object or user credentials. Since we already have an authenticated `qobuz.Session` object, we can do:

In [53]:
user_playlist_obj = qobuz.Playlist({"name": "Minim"}, session=session)
user_playlist_obj.owner = {"id": None, "name": "bbye98"} # Remove personal information.
print(user_playlist_obj)

Qobuz playlist: 'Minim' [owner: bbye98]


To make the playlist private, use `qobuz.Playlist.update()`:

In [54]:
user_playlist_obj.update(public=False)
print(f"Is '{user_playlist_obj.name}' a public playlist?", user_playlist_obj.flags["public"])

Is 'Minim' a public playlist? False


Like before, let's add some tracks to the playlist using `qobuz.Playlist.add_tracks()`. Now, the tracks to be added can be provided as Qobuz track IDs or `qobuz.Track` objects. Afterwards, the `qobuz.Playlist` object is automatically updated with the tracks' information.

In [55]:
user_playlist_obj.add_tracks([track_obj_ex.id, track_obj, album_obj.tracks[0].id, playlist_obj.tracks[14]])
for track in user_playlist_obj.tracks:
    print(track)

Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]
Qobuz track: 'Hold On & Believe (feat. The Federal Empire)' [artist: Martin Garrix, album: Hold On & Believe]
Qobuz track: 'I Ain't Going Home (feat. April Bender)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Mine' [artist: Taylor Swift, album: Speak Now]


To reorder tracks in the playlist, use `qobuz.Playlist.move_tracks()`.

:::{note}
For a given track, normally we would have to pass its *playlist track ID*, which is a track ID specific to that track in the current playlist we are working with. However, `qobuz.Playlist.move_tracks()` also accepts a `qobuz.Track` object that did not originate from another playlist. As long as the track itself is in the playlist, the `qobuz.Track` object does not need to have a `playlist_track_id` attribute.
:::

In [56]:
user_playlist_obj.move_tracks(album_obj.tracks[0], 0)
for track in user_playlist_obj.tracks:
    print(track)

Qobuz track: 'I Ain't Going Home (feat. April Bender)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]
Qobuz track: 'Hold On & Believe (feat. The Federal Empire)' [artist: Martin Garrix, album: Hold On & Believe]
Qobuz track: 'Mine' [artist: Taylor Swift, album: Speak Now]


To delete tracks from the playlist, use `qobuz.Playlist.delete_tracks()`.

In [57]:
user_playlist_obj.delete_tracks(user_playlist_obj.tracks[2])
for track in user_playlist_obj.tracks:
    print(track)

Qobuz track: 'I Ain't Going Home (feat. April Bender)' [artist: Niiko x SWAE, album: Field Trip]
Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]
Qobuz track: 'Mine' [artist: Taylor Swift, album: Speak Now]


To delete the whole playlist, use `qobuz.Playlist.delete()`. Calling this method does not delete the `qobuz.Playlist` object, but will clear all its attributes.

In [58]:
user_playlist_obj.delete()
print(user_playlist_obj)

Qobuz playlist: '<deleted>' [owner: None]


## Account- and user-related methods

There are a few remaining Qobuz API endpoints that do not fall into any of the categories above, mainly because they deal with account history and user preferences. Their implementations are practically one-to-one between `qobuz.Session` and `qobuz.User`, except `qobuz.User` generally allows `qobuz.Album`, `qobuz.Artist`, `qobuz.Playlist`, and `qobuz.Track` objects in-place of album IDs, artist IDs, playlist IDs, and track IDs, respectively, in the arguments for its methods. Additionally, `qobuz.User` methods store the requested information in easily accessible attributes, whereas `qobuz.Session` methods generally return data in JSON format. As such, we will use the authenticated `qobuz.User` instance from earlier in the rest of this tutorial.

### Search

Let's say we really like the color maroon. One would even say obsessively so. As such, we're interested in any media that contain the word "maroon". We can use `qobuz.User.search()` to query Qobuz for matching catalog items:

In [59]:
results = user.search("maroon", limit=5)

The search results is returned as a `dict`, where the keys are the media type (`"albums"`, `"artists"`, or `"tracks"`) and the values are `list`s containing Qobuz catalog information. If the `qobuz.User` object is authenticated, the search results will also include the most popular albums, artists, and tracks in `"most_popular"`:

In [60]:
for item in results["most_popular"]:
    print(item)

Qobuz artist: Maroon 5
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights]
Qobuz track: 'Marooned' [artist: Pink Floyd, album: The Division Bell]
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (3am Edition)]
Qobuz album: 'Maroon (Édition StudioMasters)' [artist: Barenaked Ladies]
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights]
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (3am Edition)]
Qobuz track: 'Marooned Jam (1994 Recording)' [artist: Pink Floyd, album: The Later Years 1987-2019]
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (3am Edition)]
Qobuz track: 'My Queen Is Nanny Of The Maroons' [artist: Sons Of Kemet, album: Your Queen Is A Reptile]
Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (The Til Dawn Edition)]
Qobuz track: 'Marooned' [artist: Pink Floyd, album: Pink Floyd - Chillout]
Qobuz track: 'Marooned Jam (1994 Recording)' [artist: Pink Floyd, album: The Later Years]
Qobuz album: 'Maroon'

If we're interested only in one media type, we can specify it in the `type` parameter. There are also boolean flags we can enable to filter for high-resolution audio (`hi_res`) and new releases (`new_release`), and to force exact matches (`strict`).

For example, if we wanted to search for albums and tracks with high-resolution audio involving artists with "maroon" in their names,

In [61]:
results_performer = user.search("maroon", type="performer", hi_res=True, limit=5)
for media_type in ("albums", "tracks"):
    print(f"{media_type.capitalize()}:")
    for item in results_performer[media_type]:
        print(f"  - {item}")
    print()

Albums:
  - Qobuz album: 'Songs About Jane' [artist: Maroon 5]
  - Qobuz album: 'V (Deluxe Version) [Deluxe]' [artist: Maroon 5]
  - Qobuz album: 'JORDI (Deluxe - Explicit) [Deluxe]' [artist: Maroon 5]
  - Qobuz album: 'Red Pill Blues (Deluxe)' [artist: Maroon 5]
  - Qobuz album: 'Hands All Over (Deluxe Edition)' [artist: Maroon 5]

Tracks:
  - Qobuz track: 'Girls Like You' [artist: Brooklyn Duo, album: Brooklyn Sessions 8]
  - Qobuz track: 'Moves Like Jagger' [artist: Maroon 5, album: Just Dance Video Game Hits, Vol. 1]
  - Qobuz track: 'This Love (Live at Live 8, Benjamin Franklin Parkway, Philadelphia, 2nd July 2005)' [artist: Maroon 5, album: Live 8 (Live, July 2005)]
  - Qobuz track: 'She Will Be Loved (Live at Live 8, Benjamin Franklin Parkway, Philadelphia, 2nd July 2005)' [artist: Maroon 5, album: Live 8 (Live, July 2005)]
  - Qobuz track: 'Girls like you' [artist: The Modo, album: Girls like you (Special Version)]



If we instead wanted to search for newly-released albums and tracks with "maroon" in the title,

In [62]:
results_tracks = user.search("maroon", type="track", new_release=True, limit=5)
for media_type in ("albums", "tracks"):
    print(f"{media_type.capitalize()}:")
    for item in results_tracks[media_type]:
        print(f"  - {item}")
    print()

Albums:
  - Qobuz album: 'Maroon' [artist: Kenji Kihara]
  - Qobuz album: 'Blue Joggers 4 Maroon Tape (#SpedUp #ChoppedUp)' [artist: JLaDonne Clothing Company]
  - Qobuz album: 'Russian Maroon' [artist: Alfin Is]
  - Qobuz album: 'Dusty Rose Joggers 4 Maroon Tape (#SpedUp #ChoppedUp)' [artist: JLaDonne Clothing Company]
  - Qobuz album: 'Maroon Hoodie 2 Green Tape (#SpedUp #ChoppedUp)' [artist: JLaDonne Clothing Company]

Tracks:
  - Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (The Til Dawn Edition)]
  - Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (The Til Dawn Edition)]
  - Qobuz track: 'Maroon Ashes' [artist: Rauelsson, album: L' Immensità]
  - Qobuz track: 'Maroon' [artist: Taylor Swift, album: Midnights (The Til Dawn Edition)]
  - Qobuz track: 'Maroon Jazz' [artist: Soft Jazz & Coffee, album: Pump up the Jazz]



### Favorites

To add albums, tracks, and artists to your favorites, use `qobuz.User.favorite_items()`:

In [63]:
user.favorite_items(albums=album_json["id"], artists=artist_json["id"], tracks=track_json_ex["id"])
for key, value in user.favorites.items():
    print(f"Favorite {key}:")
    for item in value:
        print(f"  - {item}")
    print()

Favorite albums:
  - Qobuz album: 'Field Trip' [artist: Niiko x SWAE]

Favorite artists:
  - Qobuz artist: Sick Individuals

Favorite tracks:
  - Qobuz track: 'Post Malone (feat. RANI)' [artist: Sam Feldt, album: Magnets EP]



For playlists, use `qobuz.User.favorite_playlist()`:

In [64]:
user.favorite_playlist(playlist_json["id"])
for playlist in user.playlists:
    print(playlist)

Qobuz playlist: 'Taylor Swift' [owner: Qobuz]
Qobuz playlist: 'Welcome to Qobuz' [owner: Qobuz]


To unfavorite albums, tracks, or artists, use `qobuz.User.unfavorite_items()`. For playlists, use `qobuz.User.unfavorite_playlist()`.

In [65]:
user.unfavorite_items(albums=album_json["id"], artists=artist_json["id"], tracks=track_json_ex["id"])       
user.unfavorite_playlist(playlist_json["id"])

### Curated and featured media

Every week, Qobuz prepares curated tracks and highlights featured albums and playlists depending on the user's listening history.

To get the curated tracks, use `qobuz.User.get_curated_tracks()`:

In [66]:
for track in user.get_curated_tracks(limit=5):
    print(track)

Qobuz track: 'bad guy' [artist: Billie Eilish, album: WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?]
Qobuz track: 'Skyfall' [artist: Adele, album: Skyfall (Full Length)]
Qobuz track: 'Roundabout (MQS)' [artist: Yes, album: Fragile]
Qobuz track: 'Africa (Album Version)' [artist: Toto, album: Toto IV]
Qobuz track: 'Get Lucky (feat. Pharrell Williams and Nile Rodgers)' [artist: Daft Punk feat. Pharrell Williams, album: Random Access Memories]
Qobuz track: 'Goodbye Yellow Brick Road (Remastered 2014)' [artist: Elton John, album: Goodbye Yellow Brick Road (40th Anniversary Celebration)]
Qobuz track: 'Come Together (2019 Mix)' [artist: The Beatles, album: Abbey Road (Super Deluxe Edition)]
Qobuz track: 'Hey Now' [artist: London Grammar, album: If You Wait (Deluxe)]
Qobuz track: 'Money for Nothing' [artist: Dire Straits, album: Brothers in Arms]
Qobuz track: 'Birds' [artist: Dominique Fils-Aimé, album: Nameless]
Qobuz track: 'Moondance (2013 Remaster)' [artist: Van Morrison, album: Moondance (Hi-R

To get the featured albums, use `qobuz.User.get_featured_albums()`:

In [67]:
for album in user.get_featured_albums(limit=5):
    print(album)

Qobuz album: 'Joy'All' [artist: Jenny Lewis]
Qobuz album: 'Voice of Rachmaninoff' [artist: John-Henry Crawford]
Qobuz album: 'The Age of Pleasure' [artist: Janelle Monáe]
Qobuz album: 'Weathervanes' [artist: Jason Isbell and the 400 Unit]
Qobuz album: 'Ticket to Fame' [artist: Decisive Pink]


And lastly, to get the featured playlists, use `qobuz.User.get_featured_playlists()`:

In [68]:
for playlist in user.get_featured_playlists(limit=5):
    print(playlist)

Qobuz playlist: 'New Releases' [owner: Qobuz]
Qobuz playlist: 'Jazz: 100 Cult Albums' [owner: Qobuz]
Qobuz playlist: '1990s Punk & Indie' [owner: Numero Group]
Qobuz playlist: 'Tinariwen' [owner: Qobuz]
Qobuz playlist: 'UK Newcomers' [owner: Qobuz]
